In [1]:
!pip install -U sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.5 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.2.0
    Uninstalling attrs-23.2.0:
      Successfully uninstalled attrs-23.2.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.245.0
    Uninstalling sagemaker-2.245.0:
      Successfully uninstalled sagemaker-2.245.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [sagemaker]/2 [sagemaker]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.3 which is incompatible.


In [2]:
import sagemaker
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [14]:
from sagemaker.huggingface import HuggingFace, HuggingFaceModel

In [26]:
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='src',
    instance_type='ml.g5.4xlarge',
    instance_count=1,
    role=role,
    transformers_version='4.49.0',
    pytorch_version='2.5.1',
    py_version='py311',
    base_job_name='finetune-llm',
    hyperparameters={
        'model_name': 'gpt2'
    }
)

huggingface_estimator.fit()

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: finetune-llm-2025-06-29-12-53-20-758


2025-06-29 12:53:21 Starting - Starting the training job
2025-06-29 12:53:21 Pending - Training job waiting for capacity...
2025-06-29 12:53:41 Pending - Preparing the instances for training...
2025-06-29 12:54:19 Downloading - Downloading the training image.....................
2025-06-29 12:57:51 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 550.144.03
Current installed NVIDIA driver version is 550.163.01
Skipping CUDA compat setup as newer NVIDIA driver is installed
2025-06-29 12:58:33,235 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-06-29 12:58:33,252 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-06-29 12:58:33,260 sagemaker_pytorch_container.training INFO     Block until all hos

In [27]:
model_artifact = huggingface_estimator.model_data
print("Model artifact location:", model_artifact)

Model artifact location: s3://sagemaker-eu-north-1-490004609153/finetune-llm-2025-06-29-12-53-20-758/output/model.tar.gz


In [28]:
huggingface_model = HuggingFaceModel(
    model_data=huggingface_estimator.model_data,  # output of huggingface_estimator.model_data
    role=role,
    transformers_version='4.49.0',  # or any supported version
    pytorch_version='2.6.0',        # confirmed supported
    py_version='py312',             # use python 3.10 (common)
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.2xlarge'
)

INFO:sagemaker:Creating model with name: huggingface-pytorch-inference-2025-06-29-13-01-41-333
INFO:sagemaker:Creating endpoint-config with name huggingface-pytorch-inference-2025-06-29-13-01-41-985
INFO:sagemaker:Creating endpoint with name huggingface-pytorch-inference-2025-06-29-13-01-41-985


------------!

In [ ]:
prompt = """<|im_start|>system
You are a mental health assistant. Based on the user's description, respond with a single sentence indicating the most relevant diagnosis from the mental health domain.<|im_end|>
<|im_start|>user
I feel hopeless and overwhelmed by my financial situation<|im_end|>
<|im_start|>assistant
"""

response = predictor.predict({
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 100,
        "do_sample": True,
        "temperature": 0
    }
})

generated = response[0]['generated_text']
assistant_start = generated.find("<|im_start|>assistant\n") + len("<|im_start|>assistant\n")
reply = generated[assistant_start:].strip().split("<|im_end|>")[0].strip()

print("Assistant:", reply)


Assistant: Based on what you've described, this sounds like intimacy.
